# Automated ML

Install and import dependencies

In [7]:
!pip install kaggle

If the import of data_prep fails, see the README on how to download and copy the kaggle.json

In [8]:
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Model, Webservice
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import data_prep_malware

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
ws = Workspace.from_config()
experiment_name = 'automl_experiment'

experiment=Experiment(ws, experiment_name)

dataset = data_prep_malware.get_dataset(ws)

In [10]:
dataset = data_prep_malware.create_dataset(ws)

WARNING - Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/4d18c7ab-3cf5-4b6a-844e-e9d3a4086127/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## Find or Create Compute Cluster

In [11]:
cpu_cluster_name = "cpu-cluster"

# Check if the cluster exists. If there is an error, create the cluster
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                            max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [12]:
automl_settings =  {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "featurization": 'auto'
}

automl_config = AutoMLConfig(
                    task="classification",
                    training_data=dataset,
                    label_column_name="legitimate",
                    compute_target=cpu_cluster,
                    **automl_settings)

In [9]:
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [19]:
automl_run = list(experiment.get_runs())[0]

In [20]:
best_automl_run, best_automl_model = automl_run.get_output()
best_run_metrics = best_automl_run.get_metrics()

print('Best Run Id: ', best_automl_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(best_automl_model._final_estimator)
print(best_automl_run.get_tags())

Best Run Id:  AutoML_de8e6692-dc86-44c5-8022-d4193a7e6485_38

 Accuracy: 0.9942845549447176
StackEnsembleClassifier(base_learners=[('0',
                                        Pipeline(memory=None,
                                                 steps=[('maxabsscaler',
                                                         MaxAbsScaler(copy=True)),
                                                        ('lightgbmclassifier',
                                                         LightGBMClassifier(boosting_type='gbdt',
                                                                            class_weight=None,
                                                                            colsample_bytree=1.0,
                                                                            importance_type='split',
                                                                            learning_rate=0.1,
                                                                            max

In [26]:
model = best_automl_run.register_model(model_name = 'best_automl_model', model_path = 'outputs/model.pkl', model_framework=Model.Framework.SCIKITLEARN)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In [28]:
rest_service = Model.deploy(ws, "best-model-service", [model])

rest_service.wait_for_deployment(show_output=True)
rest_service.update(enable_app_insights=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................................................
Failed


ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 783ca7ef-4a03-4f87-9407-c8500211d217
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: best-automl-model-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 5e96e4cd531a4b3983a9f3dd093c6272.azurecr.io/azureml/azureml_9febe3c381178587c812a6e9134e214b locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check th

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 783ca7ef-4a03-4f87-9407-c8500211d217
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: best-automl-model-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 5e96e4cd531a4b3983a9f3dd093c6272.azurecr.io/azureml/azureml_9febe3c381178587c812a6e9134e214b locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scor

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [ ]:
data = {"data":
        [
          {
            
          },
          {
            
          },
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

In [ ]:
import requests


response = requests.post(rest_service, 
                         headers=auth_header, 
                         json=input_data
                        )

print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = rest_service.get_logs()

for line in logs.split('\n'):
    print(line)